In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np 
import ast

In [ ]:
# load Data
df= pd.read_csv("./glowup.csv")
# print(type(df['skin_type'].iloc[0]))
# df['skin_type'].unique() # oily , normal, dry , combination, sensitive
user_skin_type= "Oily"
user_concerns = ['aging','hydration_problem']
user_ne_needed = ['Pore-Care','Anti-Aging','Hydrating','UV-Protection']

In [ ]:
# pre-filtering 
# 1> Literal Evaluation
df['skin_type'] = df['skin_type'].apply(ast.literal_eval)
df['notable_effects'] = df['notable_effects'].apply(ast.literal_eval)
print(df['skin_type'])
# print(type(df['skin_type'].iloc[0]))
df.columns
# 2> remove two columns 'Unnamed: 0.1', 'Unnamed: 0'
df=df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])




In [ ]:
# encode the skin type and notable effect
mlb_skin = MultiLabelBinarizer()
mlb_notable = MultiLabelBinarizer()


# product data
skin_matrix = mlb_skin.fit_transform(df['skin_type'])
effect_matrx = mlb_notable.fit_transform(df['notable_effects'])
product_feature = np.hstack([skin_matrix,effect_matrx])
# print(product_feature[0])

# user data
user_skin_vec = mlb_skin.transform([[user_skin_type]])
user_notable_vec = mlb_notable.transform([user_ne_needed])
# print(user_skin_vec)
# print(user_notable_vec)
user_feature = np.hstack([user_skin_vec,user_notable_vec])
print(user_feature)







In [ ]:
# calculate cosine similarity 
df['similarity_score'] = cosine_similarity(user_feature, product_feature).flatten()
print(df.columns)

In [ ]:
# create masking 
mask =df['skin_type'].apply( lambda x: user_skin_type in x)
recommendation = df[mask].copy()
print(recommendation)

In [46]:
#sort recommendation dataset by similarity score
top_recommendation = recommendation[recommendation['similarity_score']> 0].sort_values(by='similarity_score',ascending=False) 
# print(top_recommendation)



In [47]:
# for user skin type with notable effects
# output :
print(f"Top 10 recommended Product for {user_skin_type} are:\n")
print(top_recommendation[['product_name', 'brand', 'price', 'similarity_score']].head(10))

Top 10 recommended Product for Oily are:

                                          product_name          brand  price  \
76           BIODERMA Photoderm AKN Mat SPF-30 (40ml)        BIODERMA  319.0   
50                           BANANA BOAT Sport SPF110     BANANA BOAT  217.8   
138         ELSHE SKIN Anti-Aging Daily Protection 15g     ELSHE SKIN   92.0   
167              ERHA Acneact Acne Sun Friendly 30 gr            ERHA   99.0   
600  The Body Shop Tea Tree Skin Clearing Mattifyin...  THE BODY SHOP  199.0   
322          JOYLAB Brighter Sun-Day SPF 50 PA++ 50gr          JOYLAB  160.0   
125  DR JART Vital Hydra Solution Capsule Ampoule (...        DR JART  260.0   
123                       DR JART Cicapair Serum 30ml         DR JART  525.0   
121  DR JART Vital Hydra Solution Biome Essence wit...        DR JART  275.0   
119                              DR JART V7 Toner 120g        DR JART  515.0   

     similarity_score  
76           0.774597  
50           0.774597  
138  